# Hi kagglers 🙋🏻‍♂️ and Welcome to this new competition!
<div  style="width:700px; margin:0 auto;">
<img src="https://media2.giphy.com/media/flbcUFdLSHwZC03p11/200w.webp?cid=ecf05e47q1mt7k8sj3msbtpesuarrphwj0wj26w5b2p788kp&rid=200w.webp&ct=g" width="400px"/>
</div>

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
import random
from scipy import stats
import gc
from sklearn import ensemble
from sklearn.preprocessing import RobustScaler,StandardScaler,MinMaxScaler, OrdinalEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn import metrics
from lightgbm import LGBMClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import ConfusionMatrixDisplay,confusion_matrix,accuracy_score

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from sklearn import preprocessing
import seaborn as sns

import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
train = pd.read_csv('/kaggle/input/tabular-playground-series-feb-2022/train.csv')
test = pd.read_csv('/kaggle/input/tabular-playground-series-feb-2022/test.csv')
sub = pd.read_csv('/kaggle/input/tabular-playground-series-feb-2022/sample_submission.csv')

In [ ]:
# drop duplicate rows
cols = [e for e in train.columns if e not in ('row_id')]
train.drop_duplicates(subset=cols, keep='first',inplace = True)

## 1. Exploratory Data Analysis

In [ ]:
train.head()

In [ ]:
test.head()

In [ ]:
train.info()

In [ ]:
test.info()

In [ ]:
#Check if there'is null values
train.isnull().sum()

In [ ]:
#Check if there'is null values
test.isnull().sum()

## Notes : 
* We have 200k samples for training and 100k for testing
* our dataset contains 286 columns + row_id (not important) + Label(target)
* We don't have null values in our dataset 😋
* our object is to predict the "target"...

### The metric for this comptition is <b>Accuracy</b>  (one of the most common metrics used to judge the performance of classification models).
* So we are going to maximize the number of correct predictions.
<div style="width:700px; margin:0 auto;">
<img src="https://i0.wp.com/datascienceparichay.com/wp-content/uploads/2021/05/accuracy-score-v2-1.png?resize=960%2C380&ssl=1" width="400px"/>
</div>


In [ ]:
cols = [e for e in test.columns if e not in ["row_id"]]
train[cols].describe().T

In [ ]:
test[cols].describe().T

### 1. 1.  Features correlation(I will use only the first 20 Features + the target)

In [ ]:
#https://stackoverflow.com/questions/29432629/plot-correlation-matrix-using-pandas
corr = train[cols[:25]].corr()
mask = np.zeros_like(corr, dtype=bool)
mask[np.triu_indices_from(mask)] = True
corr[mask] = np.nan
(corr.style.background_gradient(cmap='coolwarm').highlight_null(null_color='#f1f1f1').set_precision(3))

In [ ]:
categorical_features = ["A0T0G0C10","A0T0G1C9","A0T0G9C1","A0T0G10C0","A0T1G0C9","A0T10G0C0","A1T0G9C0","A10T0G0C0"]
continous_features = [e for e in test.columns if e not in categorical_features + ["row_id"]]

In [ ]:
# plot the first continous 32 features 
i = 1
plt.figure()
fig, ax = plt.subplots(8, 4,figsize=(20, 22))
for feature in continous_features[:32]:
    plt.subplot(8, 4,i)
    sns.histplot(train[feature],color="blue", kde=True,bins=100, label='train_'+feature)
    sns.histplot(test[feature],color="olive", kde=True,bins=100, label='test_'+feature)
    plt.xlabel(feature, fontsize=9); plt.legend()
    i += 1
plt.show()

 ###  1. 2. Target distibution

In [ ]:
_ , ax = plt.subplots(figsize=[22,6])
sns.countplot(x="target", data=train, ax=ax)

In [ ]:
train.target.value_counts()

In [ ]:
pie, ax = plt.subplots(figsize=[18,8])
train.groupby('target').size().plot(kind='pie',autopct='%.2f',ax=ax,title='Target distibution')

* we haven't a problem of an unbalanced dataset 😋

## 2. Modeling Part

In [ ]:
le = preprocessing.LabelEncoder()
train['target'] = le.fit_transform(train['target'])

In [ ]:
param =  {  'random_state': 48,
            'n_estimators': 500, 
            'n_jobs': -1,
            }

In [ ]:
cols = [e for e in test.columns if e not in ('row_id')]

In [ ]:
preds = []
kf = StratifiedKFold(n_splits=20,random_state=48,shuffle=True)
acc=[]  # list contains accuracy for each fold
n=0
for trn_idx, test_idx in kf.split(train[cols],train['target']):
    X_tr,X_val=train[cols].iloc[trn_idx],train[cols].iloc[test_idx]
    y_tr,y_val=train['target'].iloc[trn_idx],train['target'].iloc[test_idx]
    
    model = ensemble.ExtraTreesClassifier(**param)
    model.fit(X_tr,y_tr)
    
    preds.append(model.predict(test[cols]))
    acc.append(accuracy_score(y_val, model.predict(X_val)))

    print(f"fold: {n+1} , accuracy: {round(acc[n]*100,3)}")  
    n+=1  
    
    #del X_tr,X_val,y_tr,y_val
    gc.collect()       

In [ ]:
print(f"the mean Accuracy is : {round(np.mean(acc)*100,3)} ")

* Now we have 20 arrays and each array was calculated for the i-th fold (we used 20 folds).<br>
* So I'm going to use the <b>mode</b> in order to generate the final prediction.<br>

<img src="https://k8schoollessons.com/wp-content/uploads/2019/06/Median-Mode-Mean-and-Range-1.jpg" width="450px"/>

In [ ]:
# Plot of confusion matrix for the last fold
_ , ax = plt.subplots(figsize=[16,10])
cm = confusion_matrix(model.predict(X_val),y_val)
cm_display = ConfusionMatrixDisplay(cm).plot(ax=ax)

## 3. Submission Part¶

In [ ]:
preds

In [ ]:
prediction = stats.mode(preds)[0][0]
sub['target'] = le.inverse_transform(prediction)
sub.to_csv('submission.csv', index=False)

### Post Processing : check this [Kernel](https://www.kaggle.com/hamzaghanmi/train-test-286)

In [ ]:
s1 = pd.merge(train, test, how='inner', on=cols)

In [ ]:
dic = {}
for i in range(len(s1)):
    dic[s1.loc[i]['row_id_y']] = s1.loc[i]['row_id_x']

In [ ]:
for e in dic:
    sub.loc[sub[sub['row_id']==e].index.to_list(),'target'] = train.loc[train[train['row_id']==dic[e]].index.tolist()[0],'target']

In [ ]:
sub

# To be continued... 🏂🏻
<div style="width:800px; margin:0 auto;">
<img src="https://c.tenor.com/FnQi_p4WS-sAAAAM/stay-tuned-robertidk.gif" width="500px"/>
</div>